In [42]:
import pandas as pd
!ls ..data/raw/cartchar1000.csv
df = pd.read_csv('../data/raw/cartchar1000.csv')

ls: ..data/raw/cartchar1000.csv: No such file or directory


In [43]:
df.head()

,Unnamed: 0,object,scale,angle_deg,scattered_re_0,scattered_im_0,scattered_re_1,scattered_im_1,scattered_re_2,scattered_im_2,...,scattered_re_45,scattered_im_45,scattered_re_46,scattered_im_46,scattered_re_47,scattered_im_47,scattered_re_48,scattered_im_48,scattered_re_49,scattered_im_49
0,0,bone,420.88090,55.364860,-0.984981,0.293234,0.476816,0.445085,-0.270443,-0.203460,...,0.135992,-0.126319,-0.177768,0.082180,0.015575,-0.193812,0.302563,0.409165,-0.911394,0.442634
1,1,bone,441.56550,40.486946,-0.695269,0.359409,0.041326,0.432178,0.237271,-0.227815,...,-0.012678,-0.152153,-0.018897,-0.118514,0.246970,-0.189171,0.545527,0.322201,-1.182947,0.406373
2,2,bone,433.78740,24.033340,-0.882156,0.177984,0.098830,0.478734,0.085735,-0.202439,...,0.179300,0.061936,0.105771,0.117152,0.098062,0.070257,-0.056102,0.357179,-0.966278,0.698124
3,3,hook,381.64570,63.261383,-1.157254,0.288555,-0.167163,0.297696,0.063975,-0.050759,...,-0.141187,-0.228616,0.037972,-0.118979,0.099174,-0.523578,0.128545,0.796021,-1.643938,1.248516
4,4,treasure,389.77283,65.188705,-1.066396,0.299494,0.470598,0.573193,-0.736447,-0.742074,...,-0.543234,0.652404,1.096772,-0.087111,-0.889675,-0.562728,0.596819,0.725444,-0.952794,0.250497


In [51]:
# create a neural network 
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split

X = df[df.filter(regex=("scattered*")).columns].values
y_categories = df['object']
y_unique = y_categories.unique()
num_categories = len(y_unique)
y = np.zeros(y_categories.shape[0], np.int32)
for i in range(num_categories):
    name = y_unique[i]
    y[y_categories == name] = i
    
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=567)
    
model = keras.Sequential()
model.add(keras.layers.Dense(X.shape[1]))
model.add(keras.layers.Dense(16))
model.add(keras.layers.Dense(16))
model.add(keras.layers.Dense(num_categories, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
700/700 [==============================] - 0s 318us/sample - loss: 1.2597 - acc: 0.3243
Epoch 2/100
700/700 [==============================] - 0s 61us/sample - loss: 0.8520 - acc: 0.6086
Epoch 3/100
700/700 [==============================] - 0s 61us/sample - loss: 0.6647 - acc: 0.7614
Epoch 4/100
700/700 [==============================] - 0s 62us/sample - loss: 0.5234 - acc: 0.8286
Epoch 5/100
700/700 [==============================] - 0s 63us/sample - loss: 0.4075 - acc: 0.8929
Epoch 6/100
700/700 [==============================] - 0s 81us/sample - loss: 0.3294 - acc: 0.9114
Epoch 7/100
700/700 [==============================] - 0s 67us/sample - loss: 0.2711 - acc: 0.9186
Epoch 8/100
700/700 [==============================] - 0s 64us/sample - loss: 0.2296 - acc: 0.9314
Epoch 9/100
700/700 [==============================] - 0s 60us/sample - loss: 0.1979 - acc: 0.9386
Epoch 10/100
700/700 [==============================] - 0s 65us/sample - loss: 0.1767 - acc: 0.9414
Epoch 11

Epoch 83/100
700/700 [==============================] - 0s 67us/sample - loss: 0.0048 - acc: 1.0000
Epoch 84/100
700/700 [==============================] - 0s 52us/sample - loss: 0.0047 - acc: 1.0000
Epoch 85/100
700/700 [==============================] - 0s 59us/sample - loss: 0.0042 - acc: 1.0000
Epoch 86/100
700/700 [==============================] - 0s 62us/sample - loss: 0.0039 - acc: 1.0000
Epoch 87/100
700/700 [==============================] - 0s 53us/sample - loss: 0.0035 - acc: 1.0000
Epoch 88/100
700/700 [==============================] - 0s 60us/sample - loss: 0.0036 - acc: 1.0000
Epoch 89/100
700/700 [==============================] - 0s 59us/sample - loss: 0.0034 - acc: 1.0000
Epoch 90/100
700/700 [==============================] - 0s 58us/sample - loss: 0.0031 - acc: 1.0000
Epoch 91/100
700/700 [==============================] - 0s 63us/sample - loss: 0.0031 - acc: 1.0000
Epoch 92/100
700/700 [==============================] - 0s 69us/sample - loss: 0.0029 - acc: 1.0000


In [52]:
# predict
probs = model.predict(X_test)
y_predict = np.argmax(probs, axis=1)
num_errors = np.sum(y_predict != y_test)
print(f'number of misclassifications: {num_errors} out of {len(y_test)} ({100*float(num_errors)/len(y_test):.2f} percent)')

number of misclassifications: 30 out of 300 (10.00 percent)


In [53]:
y_predict[:10]

array([2, 1, 1, 1, 2, 0, 1, 2, 1, 1])

In [54]:
y_test[:10]

array([0, 1, 1, 1, 2, 0, 1, 0, 1, 1], dtype=int32)

In [55]:
df['angle_deg'].min(), df['angle_deg'].max()

(0.22736321, 71.976685)